## B: Sử dụng API để thu thập dữ liệu
---

### ` Thư viên cần thiết` 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import os
import time
import requests
import math
import pandas as pd

### 1. Lấy giá trị của client ID từ việc inspect của trình duyệt bằng việc sử dụng selenium
- Vì trang SoundClound không cho đăng kí ClientID , ta chỉ lấy ClientID bằng cách lấy ID từ việc Inspect trình duyệt soundclound, và Client ID được cấp cho việc sử dụng trên google chrome là tạm thời
- Client ID nằm trong thẻ div với id = `g_id_intermediate_iframe`
- Ta sẽ sử dụng selenium để mở google chrome để có thể inpect lấy được file hmtl để có thể try xuất client ID

In [101]:
Client_ID = ''

In [102]:
# get path
Path = os.getcwd() + '\chromedriver.exe'
service = Service(Path)
driver = webdriver.Chrome(service=service)

# 
driver.get("https://soundcloud.com/")
L = driver.find_element(By.ID, 'g_id_intermediate_iframe')
Link = L.get_attribute('data-src')

driver.quit()

In [103]:
client_ID = Link[48:80]
client_ID

'FjnXkiGFvyaVIYtXadMm9pqIDawoxzUW'

### 2. Sử dụng API để có thể khai thác dữ liệu từ trang soundclound 
- https://api-v2.soundcloud.com/users/< entity_id>?client_id=< client_id>
- https://api-v2.soundcloud.com/users/< entity_id>/tracks?client_id=< client_id>
- https://api-v2.soundcloud.com/users/< entity_id>/playlists?client_id=< client_id>

- Ta sẽ sử dụng 3 dang API trên để khai thác dữ liệu ( users, tracks, playlists ) , Qua 3 API đó dữ liệu ta khai thác sẽ tạo được sự liên quan giữa 3 file data csv với nhau, dẫn đên sự thuận tiên cho việc phân tích.

###### a. Thiết lập các thông số cần thiết cho việc sử dụng API
- len_record: quy định số lượng user được lấy về và lưu vào users.csv
- sleep_time: thời gian chương trình sleep sau để tiếp tục thực hiện request lại vì do hết số lương requests được trong vòng 1 phút

In [105]:
len_record = 1500
# ta sẽ sleep time = 1s vì sleep time lớn sẽ làm cho chương trình chạy chậm
sleep_time = 1

###### b. Tiến hành thu thập dữ liệu và lưu dữ liệu vào 3 file (users.csv, tracks.csv, playlist.csv)

- Hàm `CollectData` dùng để gọi các API trên và thu thập dữ liệu 
  + Dữ liệu truyền vào là tên 3 file cvs sẽ được lưu thông tin theo thứ tự user, track và playlist

In [106]:
def collectData(FileUser, FileTrack, FilePlaylist):
    # Thiết lập 3 list để có thể lưu data
    List_data_user = []
    List_data_track = []
    List_data_playlist = []
    
    # tạo header để tránh lỗi 403 do requests quá nhiều lần
    headers = {'user-Agent': 'Mozilla/5.0'}
    
    Count = 0
    c = 0
    
    while (Count < len_record):
        c = c + 2
        URL_USER = f'https://api-v2.soundcloud.com/users/{c}?client_id={client_ID}'
        req_user = requests.get(URL_USER, headers=headers)
        
        if (req_user.ok == True):
            Count = Count + 1
            List_data_user.append(req_user.json())
            
            # khi thưc hiện lấy dữ liệu thành công từ user với ID = c 
            ## ta tiếp tục việc lấy data về track và playlists của user đó thông qua ID của họ
            
            #### Tracks
            URL_TRACK = f'https://api-v2.soundcloud.com/users/{c}/tracks?client_id={client_ID}'
            req_track = requests.get(URL_TRACK, headers=headers)
            
            if (req_track.ok == True):
                Data_track = req_track.json()
                
                # xóa file user trong track chỉ dữ lại user_id
                for e in Data_track["collection"]:
                    if 'user' in e:
                        del e['user']
                    
                List_data_track.extend(Data_track["collection"])
            else:
                time.sleep(sleep_time)
                
            
            #### Playlists
            URL_PLAYLIST = f'https://api-v2.soundcloud.com/users/{c}/playlists?client_id={client_ID}'
            req_playlist = requests.get(URL_PLAYLIST, headers=headers)
            
            ## xử lí vấn đề: tracks và user trong playlist
            if (req_playlist.ok == True):
                Data_playlist = req_playlist.json()
                
                # xoá dữ liệu user và tập track trong playlists
                for data in Data_playlist["collection"]:
                    # xoá data của user
                    if "user" in data:
                        del data["user"]
                    
                    # Xử lí tracks
                    if "tracks" in data:
                        _str = ""
                        for i in range(0, data["track_count"]):
                            if (i < data["track_count"] - 1):
                                _str = _str + str(data["tracks"][i]["id"]) + ","
                            else:
                                _str = _str + str(data["tracks"][i]["id"])
                        
                        del data["tracks"]
                        data["tracks"] = _str
                
                List_data_playlist.extend(Data_playlist["collection"])
                
            else:
                time.sleep(sleep_time)    
            
        else:
            time.sleep(sleep_time)
    
    # ghi dữ liệu vào file csv
    # - user.csv
    df = pd.json_normalize(List_data_user)
    df = df.reset_index()
    df.to_csv(FileUser, sep="\t", index=False, header=True, encoding='utf-8')
    
    # - track.csv
    df_track = pd.json_normalize(List_data_track)
    df_track = df_track.reset_index()
    df_track.to_csv(FileTrack, sep="\t", index=False, header=True, encoding='utf-8')
    
    #- Playlist.csv
    df = pd.json_normalize(List_data_playlist)
    df = df.reset_index()
    df.to_csv(FilePlaylist, sep="\t", index=False, header=True, encoding='utf-8')
    
    print("Done!")
            

- Chạy hàm `collectData` để tiến hành lấy dữ liệu và lưu vào 3 file csv

In [ ]:
collectData("user.csv", "track.csv", "playlist.csv")

- Xem kết quả trong file `user.csv`: 5 records đầu tiên

In [2]:
users = pd.read_csv('user.csv', sep='\t')
users.head()

,index,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,description,followers_count,followings_count,...,station_permalink,creator_subscription.product.id,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking,badges.pro,badges.pro_unlimited,badges.verified,visuals
0,0,https://i1.sndcdn.com/avatars-LQHwEEFP8SciWWEl...,Berlin,384,DE,2007-07-20T16:07:00Z,[{'product': {'id': 'creator-pro-unlimited'}}],Founder & CEO Dance. Founder SoundCloud and ar...,45706,2000,...,artist-stations:2,creator-pro-unlimited,soundcloud:users:2,True,"[{'urn': 'soundcloud:visuals:105136722', 'entr...",NaN,False,True,False,NaN
1,1,https://i1.sndcdn.com/avatars-000019102368-0eu...,Stockholm,31,DE,2007-07-20T17:54:00Z,[{'product': {'id': 'free'}}],"Opps, I can change Johans description. /emil",8325,139,...,artist-stations:6,free,NaN,NaN,NaN,NaN,False,False,False,NaN
2,2,https://i1.sndcdn.com/avatars-000208970259-vng...,Berlin/NYC,697,DE,2007-07-25T23:08:00Z,[{'product': {'id': 'creator-pro-unlimited'}}],"Hi, my name is Alex. I like Sound. Founder & C...",180218,1710,...,artist-stations:8,creator-pro-unlimited,soundcloud:users:8,True,"[{'urn': 'soundcloud:visuals:3142', 'entry_tim...",NaN,False,True,False,NaN
3,3,https://a1.sndcdn.com/images/default_avatar_la...,Stockholm,5,SE,2007-07-27T14:15:18Z,[{'product': {'id': 'free'}}],NaN,1246,51,...,artist-stations:10,free,NaN,NaN,NaN,NaN,False,False,False,NaN
4,4,https://i1.sndcdn.com/avatars-YU1Y4hVJyNPwYfRI...,NaN,506,NaN,2007-07-27T14:18:00Z,[{'product': {'id': 'creator-pro-unlimited'}}],Tell us about Youssef!,23034,459,...,artist-stations:12,creator-pro-unlimited,NaN,NaN,NaN,NaN,False,True,False,NaN


- Xem kết quả trong file `track.csv`: 5 records đầu tiên

In [ ]:
tracks = pd.read_csv('track.csv', sep='\t')
tracks.head()

- Xem kết quả trong file `playlist.csv`: 5 records đầu tiên

In [15]:
playlists = pd.read_csv('playlist.csv', sep='\t')
playlists.head()

,index,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,...,tag_list,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,tracks
0,0,NaN,2021-08-07T12:14:56Z,NaN,23088711,all,NaN,1297775500,playlist,NaN,...,NaN,Tent session,https://api.soundcloud.com/playlists/1297775500,16,NaN,False,2021-08-07T12:19:49Z,2021-08-07T12:19:49Z,3,"272612733,649886222,425592600"
1,1,NaN,2016-05-12T20:54:09Z,NaN,36916723,all,NaN,224489873,playlist,NaN,...,NaN,Maui,https://api.soundcloud.com/playlists/224489873,16,NaN,False,NaN,2016-05-12T20:54:09Z,5,"259629933,173516843,237681279,172743281,177856481"
2,2,NaN,2013-05-02T12:03:59Z,NaN,14081758,all,NaN,5316318,playlist,NaN,...,NaN,All Drankensteins,https://api.soundcloud.com/playlists/5316318,16,NaN,False,NaN,2013-05-02T12:03:59Z,4,"23876268,22746957,39007821,59622690"
3,3,NaN,2008-07-22T13:28:59Z,NaN,538434,none,NaN,315,playlist,NaN,...,NaN,All of my tracks,https://api.soundcloud.com/playlists/315,16,NaN,False,NaN,2008-07-22T13:28:59Z,2,"19,7793"
4,4,NaN,2016-12-11T18:17:24Z,NaN,7217789,all,NaN,282653735,playlist,NaN,...,NaN,Ethiopiques,https://api.soundcloud.com/playlists/282653735,20,NaN,False,2016-12-11T18:17:24Z,2016-12-11T18:17:24Z,13,"31529637,95060900,125486033,61689615,8547837,3..."


---


### 3. Phân tích và đánh giá dữ liệu lấy được từ việc sử dụng API 

##### A . Mô tả dữ liệu lấy được

##### `user.csv`

- **Kiểm tra số lương `mẫu và thuộc tính`**

In [3]:
users = pd.read_csv('user.csv', sep='\t')
users = users.drop(columns="index")
users.shape

(1500, 38)

- **Phân tích dữ liệu lấy đươc**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa  
|:--------:|:-------------:|:-----------:
|**avatar_url**|Chuỗi |Đường dẫn đến ản đại diện của user|
|**created_at**|Thời gian|Thời gian khởi tạo user|   
|**description**|Chuỗi|Mô tả user của người đăng tải| 
|**city**|Chuỗi|Thành phố user đang sống|
|**country_code**|Chuỗi|Mã quốc gia user đang sống|
|**id**|Kiểu phân loại|Định danh của user|
|**comments_count**|Số nguyên|Số lượt bình luận của user|
|**likes_count**|Số nguyên|Số lượt được yêu thích của user|
|**followers_count**|Số nguyên|Số lượt theo dõi của user|
|**followings_count**|Số nguyên|Số lượt user theo dõi|
|**playlist_likes_count**|Số nguyên|Số lượt được yêu thích các playlist của user|
|**playlist_count**|Số nguyên|Số playlist của user|
|**reposts_count**|Số nguyên|Số bài đăng lại của user|
|**track_count**|Số nguyên|Số lượng track của user|
|**kind**|Chuỗi|Loại đối tượng|
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng| 
|**first_name**|Chuỗi|Tên của user|   
|**last_name**|Chuỗi|Họ của user|  
|**full_name**|Chuỗi|Tên đầy đủ của user| 
|**username**|Chuỗi|Tên đăng nhập của user|   
|**groups_count**|Số nguyên|Số nhóm của user|
|**permalink**|Chuỗi|Đường dẫn cố định của tài nguyên| 
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |   
|**uri**|Chuỗi|Đường dẫn API của user|    
|**url**|Chuỗi|Đường dẫn API của user|    
|**urn**|Chuỗi|Đường dẫn API của user|    
|**creator_subscriptions**|Chuỗi|Loại thành viên của user|    
|**creator_subscription.product.id**|Chuỗi|Loại thành viên của user|    
|**badges.pro_unlimited'**|Luận lý|User có phải người dùng pro_unlimited không|
|**verified**|Luận lý|Xác thực người dùng|
|**visuals**|Số thực||
|**visuals.urn**|Chuỗi
|**visuals.enabled**|Chuỗi|
|**visuals.visuals**|Chuỗi|
|**visuals.tracking**|Số thực|

##### `track.csv`

- **Kiểm tra số lương `mẫu và thuộc tính`**

In [4]:
tracks = pd.read_csv('track.csv', sep='\t')
tracks = tracks.drop(columns="index")
tracks.shape

(4535, 63)

- **Phân tích dữ liệu lấy đươc**

In [8]:
## chua lam

##### `playlist.csv`

- **Kiểm tra số lương `mẫu và thuộc tính`**

In [5]:
playlists = pd.read_csv('playlist.csv', sep='\t')
playlists = playlists.drop(columns="index")
playlists.shape

(2093, 32)

- **Phân tích dữ liệu lấy đươc**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa của dữ liệu
|:-----------:|:----------------:|:---------:|
|**artwork_url**|Chuỗi |URL ảnh của playlist|
|**created_at**|Thời gian|Thời gian khởi tạo playlist|  
|**description**|Chuỗi|Mô tả của người đăng tải| 
|**duration**|Số nguyên|Tổng thời gian các track nhạc trong playlist|
|**embeddable_by**|Chuỗi|
|**genre**|Chuỗi|Thể loại nhạc của playlist|
|**id**|Kiểu phân loại|Định danh cho playlist|
|**kind**|Chuỗi|Loại user|
|**label_name**|Chuỗi||   
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng của tác giả| 
|**license**|Chuỗi|Giấy phép|
|**likes_count**|Số nguyên|Số lượt yêu thích của playlist|
|**managed_by_feeds**|Luận lý|Playlist có được quản lý bởi Feed|
|**permalink**|Chuỗi|  
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |  
|**public**|Luận lý|Playlist có công khai hay không?|
|**purchase_title**|Chuỗi|     
|**purchase_url**|Chuỗi|       
|**release_date**|Thời gian|Thời gian xuất bản playlist|   
|**reposts_count**|Số nguyên|Số lượt đăng lại playlist|
|**secret_token**|Chuỗi|
|**sharing**|Chuỗi|Chế độ chia sẻ công khai hay riêng tư|
|**tag_list**|Chuỗi|
|**title**|Chuỗi|Tiêu đề playlist|    
|**uri**|Chuỗi|Đường dẫn API của playlist|    
|**user_id**|Kiểu phân loại|Định danh của người sở hữu playlist|   
|**set_type**|Chuỗi|
|**is_album**|Luận lý|Playlist có phải album hay không|
|**published_at**|Thời gian|Thời điểm đăng tải playlist|   
|**display_date**|Thời gian|Thời điểm playlist hiển thị|   
|**track_counts**|Số nguyên|Số lượng track trong playlist|
|**tracks**|Chuỗi|Danh sách id của các track trong playlist được ngăn cách bởi ","|

#### B. Loại bỏ 1 số thông tin không cần thiết

- Trong `Playlist`
 + Qua việc phân tích các thuộc tính lấy được từ file `playlists.cvs` , ta nhìn thấy được 2 thuộc được đưa vào nghi vấn dữ liệu ở 2 cột đó có thể trùng nhau nếu nó thật sự trùng nhau ta có thể loại bỏ bớt 1 cột để dữ liệu tránh bị bias
    + `created_at`: Thời gian khởi tạo playlist
    + `published_at`: Thời điểm đăng tải playlist

In [98]:
# tiến hành kiểm tra dữ liệu
count = 0
count_none_createdAt = 0
count_none_published_at = 0
count_sam = 0

for i in range(0, playlists.shape[0]):
    if (type(playlists["created_at"][i]) == float):
        count_none_createAt += 1
    if (type(playlists["published_at"][i]) == float):
        count_none_published_at += 1
    
    if (not(type(playlists["published_at"][i]) == float) and not(type(playlists["created_at"][i]) == float)):
        if (playlists["created_at"][i] == playlists["published_at"][i]):
            count_sam = count_sam + 1
    

In [99]:
print("Giá trị bị rỗng ở created_at:", count_none_createdAt ) 
print("Giá trị bị rỗng ở published_at:", count_none_published_at)
print("Giá trị trùng lặp giữa 2 thuộc tính:", count_sam)

Giá trị bị rỗng ở created_at: 0
Giá trị bị rỗng ở published_at: 86
Giá trị trùng lặp giữa 2 thuộc tính: 33


 - Qua 3 thông tin dữ liệu trên ta thấy được dữ liệu trong cột `published_at` thường rơi vào rỗng hoặc bị trùng lặp với cột `created_at` => Ta sẽ xóa bỏ đi cột `published_at` trong file `playlists.csv`

##### C .Mối quan hệ giữa 3 file user, track và playlist trong csv

![image info](picture.png)